In [3]:
using LinearAlgebra

In [27]:
function upper_hessenberg_form(A)
# A = [1. 2 3 4 5 6; 7 8 9 10 11 12; 13 14 15 16 17 18; 19 20 21 22 23 24; 25 26 27 28 29 30; 31 32 33 34 35 36]
# display(A)
  n = size(A, 1)
  # k=1
  for k = 1:n-2
    beta = maximum(abs.(A[k+1:n, k]))
    if beta != 0
      ### Setting up Qk
      A[k+1:n, k] /= beta
      tao = norm(A[k+1:n, k], 2) # norm() is the euclidean norm
      if A[k+1, k] < 0 
        tao = - tao
      end
      eta = A[k+1, k] + tao
      A[k+1, k] = 1
      A[k+2:n, k] /= eta
      gamma = eta/tao
      tao *= beta
      ###

      ### Multiply on the left by Qk
      b = transpose(A[k+1:n, k]) * A[k+1:n, k+1:n]
      b *= - gamma
      A[k+1:n, k+1:n] += (A[k+1:n, k] * b)  
      ###

      ### Multiply on the right by Qk
      b = A[:, k+1:n] * A[k+1:n, k]
      b *= -gamma 
      A[:, k+1:n] += (b * transpose(A[k+1:n, k]))
      ###   

      A[k+1,k] = - tao
    end
  end
  return A
end

upper_hessenberg_form (generic function with 1 method)

In [38]:
A = [1. 50 -3 4; 5 6 -70 8; 9 -10 111 12; 130 .14 15 0.0016]
B = [1. 50 -3 4; 5 6 -70 8; 9 -10 111 12; 130 .14 15 0.0016]
display(A)
C = hessenberg!(A)
D = upper_hessenberg_form(B)
show(C)
display(D)


4×4 Array{Float64,2}:
   1.0   50.0    -3.0   4.0   
   5.0    6.0   -70.0   8.0   
   9.0  -10.0   111.0  12.0   
 130.0    0.14   15.0   0.0016

4×4 Array{Float64,2}:
    1.0        -5.69754    -5.20226  -49.6535 
 -130.407       2.49612    19.4482    -3.75938
    0.0664663  19.4012     92.0192   -25.5032 
    0.960068   -0.120091  -85.9734    22.4862 

Hessenberg{Float64,Array{Float64,2}}([1.0 -5.69754 -5.20226 -49.6535; -130.407 2.49612 19.4482 -3.75938; 0.0664663 19.4012 92.0192 -25.5032; 0.960068 -0.120091 -85.9734 22.4862], [1.03834, 1.97157, 0.0])